In [ ]:
import os
from dotenv import load_dotenv
from huggingface_hub import InferenceClient
from langchain_core.prompts import PromptTemplate


In [ ]:
load_dotenv()
HF_TOKEN = os.getenv("HUGGINGFACE_TOKEN")

In [15]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      

      Question: {question}
    """,
    input_variables = ['question']
)

question= "What is the capital of Brazil?"
final_prompt = prompt.invoke({"question": question}).to_string()

In [ ]:

client = InferenceClient()
llm="meta-llama/Llama-3.1-8B-Instruct"
completion = client.chat.completions.create(
    model=llm,
    messages=[
        {"role": "user", "content": final_prompt}
    ]
)

print(completion.choices[0].message.content)

The capital of Brazil is Brasília! Would you like to know more about Brazil or is there anything else I can assist you with?
